In [43]:
# Character level lyrics generation using RNNs (LSTM)
import sys, os, random, string
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import string

import CharlyricsDataset
from RNN import RNN
import glob

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# interactive mode
plt.ion()

from pathlib import Path
from config import config
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
train_dataset = CharlyricsDataset.CharLyricsDataset(config.DATA.LYRICS, config.TRAIN.MAX_LEN)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.TRAIN.BATCH_SIZE,
    shuffle=False,
    drop_last=True,
    num_workers=1
)

In [46]:
model = RNN(utils.get_total_characters(), config.TRAIN.HIDDEN_SIZE, config.TRAIN.LSTM_N_LAYERS, utils.get_total_characters()).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.TRAIN.LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

In [51]:
for epoch in range(config.TRAIN.EPOCHS):
    total_loss = 0
    print(f"Training Epoch {epoch}")
    for _, batch in enumerate(train_loader):
        model.zero_grad()
        input_seq, output_seq = batch

        input_seq = input_seq.to(device)
        output_seq = output_seq.to(device)
        loss = 0

        # vectorize this
        for c in range(config.TRAIN.MAX_LEN):
            output = model(input_seq[:, c])
            loss += loss_fn(output, output_seq[:, c])
            total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Total Loss {total_loss/(config.TRAIN.MAX_LEN * len(train_loader))}")

Training Epoch 0
Total Loss 113.32127783965319
Training Epoch 1
Total Loss 113.29228707451374
Training Epoch 2
Total Loss 113.26473779741674
Training Epoch 3
Total Loss 113.23847838480026
Training Epoch 4
Total Loss 113.21343167554588
Training Epoch 5
Total Loss 113.1894790230319
Training Epoch 6
Total Loss 113.16661517336965
Training Epoch 7
Total Loss 113.14476598206907
Training Epoch 8
Total Loss 113.12390672866255
Training Epoch 9
Total Loss 113.1039573180303


'\x0c'